In [6]:
from datetime import datetime, timedelta, timezone
import threading
import time 


In [7]:
class Scheduler:
    """
    A class to represent a schedule for a job with a start and end date.
    
    Attributes:
    -----------
    stdate : str or datetime
        The start date and time of the schedule. If not provided, defaults to the current date and time.
    time_zone : str
        The timezone of the schedule. Defaults to the current timezone of the computer.
    
    Methods:
    --------
    add_job():
        Adds a job for running it 
    run_all():
        Runs all jobs with ot without threading
        
    Raises:
    -------
    ValueError:
        If Startdate is less than the current date.
    """

    def __init__(self, threading=False, startdate=None, time_zone=None):
        self.threads = threading
        self.stdate = startdate if startdate else datetime.now().strftime('%Y-%m-%d %H:%M')
        self.stdate = datetime.strptime(self.stdate, '%Y-%m-%d %H:%M')
        self.jobs = []

        if time_zone is not None:
            self.time_zone = timezone(timedelta(hours=time_zone))
        else:
            # use the system's local timezone if no offset is provided
            self.time_zone = datetime.now().astimezone().tzinfo

        # checking if startdate is less than the current date
        if self.stdate.strftime('%Y-%m-%d %H:%M') < datetime.now().strftime('%Y-%m-%d %H:%M'):
            raise ValueError("Startdate cannot be less than the current date")
        
        # counting time for specified timezone 
        startdate_with_timezone = self.stdate.astimezone(self.time_zone)




############################# METHODS #############################
    
    ####### add_job ####### 
    def add_job(self, job):
        # if the job doesn't have a startdate, use the scheduler's startdate
        if job.startdate is None:
            job.startdate = self.stdate
            print('adding timefromstart {}'.format(job.startdate))
            
        # if the job doesn't have a time_zone, use the scheduler's time_zone
        if job.time_zone is None:
            job.time_zone = self.time_zone
            # print('adding timezone {}'.format(job.time_zone))
        # adding jobs 
        self.jobs.append(job)
        return self


    ####### run_all ####### 
    def run_all(self):
        
        # running jobs simultaneously when threads is true
        if self.threads:
            threads = []
            for job in self.jobs:
                # calling the Job run method for each thread
                thread = threading.Thread(target=job.run)
                threads.append(thread)
                
                # starting the execution of the thread
                thread.start()
            
            # ensure that run_all doesn't finish until all jobs have completed their execution.
            for thread in threads:
                thread.join()
        else:
            for job in self.jobs:
                job.run()
                


In [14]:
# Example Job Class
class Job:
    """
    A class to run jobs with specified time 
    
    Attributes:
    -----------
    startdate : str or datetime
        The start date and time of job. If not provided, taken from Chedule class.
    time_zone : str
        The timezone of the schedule. If not provided taken from Chedule class
    
    Methods:
    --------
    do():
        finds the job for running 
    run_all():
        Runs all jobs with ot without threading
        
    Raises:
    -------
    ValueError:
        If Startdate is less than the current date.
    """
    def __init__(self, startdate =None, time_zone=None):
        self.startdate = startdate # starting date of shcedule
        self.time_zone =time_zone # timezone hour offset
        self.func = None  # function
        self.name = None # name of function
        self.unit = None # time unit
        self.enddate = None # until which time does user wants to run job 
        self.interval = None # interval between running jobs
        self.next_run = None #next run time
        self.args = ()  # Positional arguments to pass to the job function.
        self.kwargs = {}  # Keyword arguments to pass to the job function.
        

        # setting time_zone 
        if time_zone is not None:
            self.time_zone = timezone(timedelta(hours=time_zone))
        if isinstance(self.startdate, str):
                # Converting the string to a datetime object
                self.startdate = datetime.strptime(self.startdate, '%Y-%m-%d %H:%M')
                self.startdate = self.startdate.astimezone(self.time_zone)
                print(self.startdate)
            # Checking if startdate is less than the current date
                if self.startdate.strftime('%Y-%m-%d %H:%M') < datetime.now().strftime('%Y-%m-%d %H:%M'):
                    raise ValueError("Job startdate cannot be less than the current date")

            


    ############################# METHODS #############################
    
    #### do ###
    def do(self, func, name):
        """
        Parameters
        ----------
        func : function
            The function to be searched and run.
        name : str
            The name of the function provided by the user.
        """
        self.func = func
        self.name = name
        return self
    
    #### every ###
    def every(self, interval):
        """
        Parameters
        ----------
        interval : int
            interval in which code will run (used to calculate next run time) 
        """
        self.interval = interval
        return self

    #### until ###
    def until(self, enddate):
        """
        Parameters
        ----------
        enddate : str
            The date and time until which the user wants to run the code. The format should be 'YYYY-MM-DD HH:MM:SS'.

        Raises
        ------
        ValueError
            If the startdate occurs after enddate
        """
        # self.startdate = self.startdate.astimezone(self.time_zone)
        if isinstance(enddate, str):
            # Converting the string to a datetime object
            enddate = datetime.strptime(enddate, '%Y-%m-%d %H:%M')

        # startdate and enddte for specific timezones 
        # self.startdate = self.startdate.astimezone(self.time_zone)
        self.enddate = enddate.astimezone(self.time_zone)

        # raising approptiate error
        if self.enddate <= self.startdate:
            raise ValueError("Startdate cannot be more than or equal to the end date")

        return self
    
    #### second ###
    @property
    def second(self):
        self.unit = 'second'
        return self

    #### calc_next_run ###
    def calc_next_run(self, current_time):
        """
        Calculates the next run time based on the interval and unit.
        
        Parameters
        ----------
        current_time : datetime
            The current time from which the next run is calculated.
        
        Returns
        -------
        datetime
            The next run time.
        """
        if self.unit == 'second':
            next_run = current_time + timedelta(seconds=self.interval)
            if self.enddate and next_run > self.enddate:
                return None  # Indicates the job should stop
            return next_run
        
    
    
    # რეალური რანის ფუნქცია მაქვს დასაწერი !!!!!!!!!!!

    def run(self):
        # searching if there are functions to run 
            if self.func is not None:
                current_time = datetime.now().astimezone(self.time_zone)
                self.startdate = self.startdate.astimezone(self.time_zone)
                if current_time.strftime('%Y-%m-%d %H:%M') < self.startdate.strftime('%Y-%m-%d %H:%M'):
                    wait_time = (self.startdate - current_time).total_seconds()
                    print(f"Waiting for {wait_time} seconds until startdate.")
                    time.sleep(wait_time)
                    current_time = self.startdate
                elif current_time.strftime('%Y-%m-%d %H:%M') > self.startdate.strftime('%Y-%m-%d %H:%M'):
                    raise ValueError("WRONG")
                while True:
                    self.func(*self.args, **self.kwargs)
                    current_time = self.calc_next_run(current_time)

                    if current_time is None:
                        print("Job has ended.")
                        break

                    print(f"Next run at: {current_time}")
                    time.sleep((current_time - datetime.now().astimezone(self.time_zone)).total_seconds())
            else:
                print('No function assigned to run.')



In [15]:
# Example functions
def report1():
    print("Function report1 is running.")

def report2():
    print("Function report2 is running.")

def report3():
    print("Function report3 is running.")

# Create Job instances and associate functions with them
job1 = Job().do(report1, 'report1').every(2).second.until('2024-08-16 10:00')
# job2 = Job(startdate='2024-08-15 10:00',time_zone=5).do(report2, 'report2')
# job3 = Job(startdate='2024-08-15 10:00',time_zone=10).do(report3, 'report3').every(2).second.until('2024-08-16 10:00')


# .every(2).second.until('2024-08-14 16:42')
# Now you can add these jobs to the Scheduler and run them
scheduler = Scheduler(threading=False)
scheduler.add_job(job1)
# scheduler.add_job(job2)
# scheduler.add_job(job3)
scheduler.run_all()

TypeError: '<=' not supported between instances of 'datetime.datetime' and 'NoneType'